In [54]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install geopy

                                              0.0/125.4 kB ? eta -:--:--
     ---------                               30.7/125.4 kB 1.4 MB/s eta 0:00:01
     ---------------------------           92.2/125.4 kB 871.5 kB/s eta 0:00:01
     -------------------------------------- 125.4/125.4 kB 1.1 MB/s eta 0:00:00
                                              0.0/40.3 kB ? eta -:--:--
     ---------------------------------------- 40.3/40.3 kB 1.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [37]:
import requests
from bs4 import BeautifulSoup
import re
import json
from geopy.geocoders import Nominatim

def obtener_coordenadas(ciudad, pais):
    geolocalizador = Nominatim(user_agent="myGeocoder")
    direccion = f"{ciudad}, {pais}"
    ubicacion = geolocalizador.geocode(direccion)
    
    if ubicacion:
        return ubicacion.latitude, ubicacion.longitude
    else:
        return None, None

def extraer_datos_ciudades(soup):
    ciudades = {"type": "FeatureCollection", "features": []}

    for ciudad_element in soup.find_all('h3'):
        nombre_ciudad_match = re.search(r'<a.*?>(.*?)<\/a>', str(ciudad_element))
        nombre_ciudad = nombre_ciudad_match.group(1) if nombre_ciudad_match else "No encontrado"

        pais_match = re.search(r'<a.*?>(.*?)<\/a>,\s(.*?)\s\(', str(ciudad_element))
        pais = pais_match.group(2) if pais_match else "No encontrado"

        poblacion_element = ciudad_element.find_next('p', string=re.compile('Población 2023'))
        poblacion_2023 = ""
        if poblacion_element:
            next_element = poblacion_element.find_next_sibling('p')
            if next_element:
                poblacion_text = next_element.get_text()
                poblacion_2023_match = re.search(r'([\d,]+)\s+\(Variación', poblacion_text)
                poblacion_2023 = poblacion_2023_match.group(1).replace(",", "") if poblacion_2023_match else "No se encontró"

        if nombre_ciudad != "No encontrado" and pais != "No encontrado":
            latitud, longitud = obtener_coordenadas(nombre_ciudad, pais)
            ciudad = {
                "type": "Feature",
                "geometry": {
                    "type": "Point",
                    "coordinates": [longitud, latitud]
                },
                "properties": {
                    "Nombre": nombre_ciudad,
                    "País": pais,
                    "Población 2023": poblacion_2023
                }
            }
            ciudades["features"].append(ciudad)
    
    return ciudades

def guardar_datos_json(ciudades):
    with open("datos_ciudades.json", "w", encoding="utf-8") as geojson_file:
        json.dump(ciudades, geojson_file, ensure_ascii=False, indent=4)

def main():
    url = "https://www.archdaily.cl/cl/1003731/estas-son-las-ciudades-mas-pobladas-de-america-latina-en-2023"

    response = requests.get(url)
    html_content = response.text
    soup = BeautifulSoup(html_content, 'html.parser')

    ciudades = extraer_datos_ciudades(soup)
    guardar_datos_json(ciudades)

if __name__ == "__main__":
    main()


In [34]:
import requests
import re

url = "https://www.archdaily.cl/cl/1003731/estas-son-las-ciudades-mas-pobladas-de-america-latina-en-2023"

# Obtener el contenido HTML de la página
response = requests.get(url)
html_content = response.text

# Expresión regular para extraer la información de población
city_population_pattern = r'<h3>\d+° - <a href=".*?">(.*?)<\/a>,.*?<strong>Población 2023: <\/strong>(.*?)\s.*?'

# Encontrar coincidencias en el HTML
matches = re.findall(city_population_pattern, html_content, re.DOTALL)

# Lista para almacenar los datos de las ciudades
ciudades = []

# Función para limpiar la información de población
def clean_population_info(population):
    population = population.replace(",", "")  # Eliminar comas
    population = re.sub(r'<[^>]*>', '', population)  # Eliminar etiquetas HTML
    return population.strip()

# Iterar sobre las coincidencias encontradas
for match in matches:
    city_name = match[0]
    population_2023 = clean_population_info(match[1])

    ciudad_data = {
        "Nombre": city_name,
        "Población 2023": population_2023,
    }
    ciudades.append(ciudad_data)

# Imprimir los resultados obtenidos con expresiones regulares
for ciudad in ciudades:
    print(f"Nombre de la ciudad: {ciudad['Nombre']}")
    print(f"Población en 2023: {ciudad['Población 2023']}")
    print("--------------------------------------")


Nombre de la ciudad: Sao Paulo
Población en 2023: 22619736
--------------------------------------
Nombre de la ciudad: Ciudad de México
Población en 2023: 22281442
--------------------------------------
Nombre de la ciudad: Buenos Aires
Población en 2023: 15490415
--------------------------------------
Nombre de la ciudad: Río de Janeiro
Población en 2023: 11507960
--------------------------------------
Nombre de la ciudad: Lima
Población en 2023: 6903392
--------------------------------------
Nombre de la ciudad: Belo Horizonte
Población en 2023: 6247889
--------------------------------------
Nombre de la ciudad: Guadalajara
Población en 2023: 5419880
--------------------------------------
Nombre de la ciudad: Monterrey
Población en 2023: 5116647
--------------------------------------
